In [ ]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes==0.41.1
!pip install -U transformers accelerate

Found existing installation: bitsandbytes 0.46.0
Uninstalling bitsandbytes-0.46.0:
  Successfully uninstalled bitsandbytes-0.46.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [ ]:
from google.colab import drive
import pandas as pd
import json
from typing import List
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import pipeline
from tqdm import tqdm

from transformers.utils import logging
logging.set_verbosity_error()

In [ ]:
# Monta Google Drive
drive.mount('/content/drive')

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


In [ ]:
csv_path = '/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv'

df = pd.read_csv(csv_path)

if "Qwen" not in df.columns:
    df["Qwen"] = ""

In [ ]:
#from huggingface_hub import login
#login() #login with your token in read-mode

In [ ]:
model_name="Qwen/Qwen2.5-3B-Instruct"

#load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto", #fp16
    device_map="auto" #gpu
)

#load the tokenizer associated to the model
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# QWEN (few shot)

Function that define the prompt and keep only the generated response (no use of text-generator)

In [ ]:
#Build a conversation turn prompt with instructions and few-shot examples
def translate_old_to_modern_italian(sentence, model, tokenizer):
    messages = [
        {
            "role": "system",
            "content": "Sei un traduttore e filologo italiano. Traduci dall'italiano antico (Fiorentino, 1200-1400) all'italiano moderno."
            "- Il significato della frase originale deve essere rispettato senza omissioni o aggiunte improprie.\n"
            "- La frase deve essere grammaticalmente corretta (tempi verbali, punteggiatura, accordi morfologici come genere, numero e concordanza)\n"
            "- Il lessico e la struttura sintattica devono essere naturali e adatti a un lettore contemporaneo italiano.\n"
            "- I termini arcaici e non piu in uso devono essere sostituiti con termini moderni appropriati.\n"
            "- Il tono e lo stile devono essere moderni.\n"
            "- I riferimenti storici, culturali o semantici devono essere interpretati correttamente e adattati al contesto. \n\n"
        },
        {
            "role": "user",
            "content": "Traduci: E però che la seconda suole talora per la grande provedenza fare timoroso, e la prima per l’ardire rendere altrui matto, assai utile è la presente distinzione."
        },
        {
            "role": "assistant",
            "content": "Poiché la seconda, per la sua grande previdenza, può talora incutere timore, e la prima, per l’ardire, condurre alla follia, questa distinzione si rivela assai utile."
        },
        {
            "role": "user",
            "content":  "Traduci: Niuna cosa si dimostra onesta che con suo contrario non si disonesti."
        },
        {
            "role": "assistant",
            "content": "Nessuna cosa si mostra onesta, se non si disonora attraverso il confronto con il suo contrario."
        },
        {
            "role": "user",
            "content":  "Traduci: Era una giovane di maravigliosa bellezza e di costumi onesti, la quale con leggiadria e onestà si portava."
        },
        {
            "role": "assistant",
            "content": "Era una giovane di straordinaria bellezza e di buoni costumi, che si comportava con eleganza e compostezza."
        },
        {
            "role": "user",
            "content":  "Traduci: Diffinizione d'una cosa è dicere ciò che quella cosa è, per tali parole che non si convegnano ad un'altra cosa."
        },
        {
            "role": "assistant",
            "content": "La definizione di una cosa consiste nel dire ciò che essa è, usando parole che non si adattino a un'altra cosa."
        },
        {
            "role": "user",
            "content": f"Traduci: {sentence}"
        },
    ]


    #We build the final prompt in the form required by the model using "apply_chat_template":
    #<|im_start|>system
    #Sei un traduttore...<|im_end|>
    #<|im_start|>user
    #Traduci: E però che la seconda...<|im_end|>
    #<|im_start|>assistant
    #..........
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    #tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        #the following block generate the model output
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            #temperature=0.7,
            #top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id, #padding token to uniform the batch
            eos_token_id=tokenizer.eos_token_id #end of sentence token
        )

    #let's keep only the generated part and remove the prompt input tokens:
      #outputs[0] -> total output (prompt +  response)
      #[inputs['input_ids'].shape[1]:] -> number of tokens in the original prompt
    generated_ids_trimmed = outputs[0][inputs['input_ids'].shape[1]:]

    #convert numerical tokens into a string
    response = tokenizer.decode(generated_ids_trimmed, skip_special_tokens=True).strip()
    return response


In [ ]:
# Translate the sentences
for idx, row in df.iterrows():
    sentence = row["Sentence"]
    translation = translate_old_to_modern_italian(sentence, model, tokenizer)
    df.at[idx, "Qwen"] = translation
    print(f"Translated row {idx + 1}/{len(df)}")
    print(translation)

df.to_csv("dataset_with_qwen(few-shot).csv", index=False)

print("End of processing")

Translated row 1/97
Quella guerra era l'opera perché etc. Dall'altra parte, Aiace era un cavaliere francese e valoroso con l'arma, dotato di grande abilità, ma non era ricco di grande saggezza.
Translated row 2/97
Crudele, prende vendetta di tutte le colpe, come stabilisce la legge, e perdonerà a nessun cavaliere che pecca.
Translated row 3/97
Non era ornato da una forza di animo superiore Ponio Aufidiano, cavaliere romano.
Translated row 4/97
Se questo piace a tutti et se il tempo ha bisogno di avere Pompeio come cavaliere e non come compagno, non riterrò più i fatti.
Translated row 5/97
L'ufficio di questa arte sembra consistere nel dire appositamente per far credere, il fine è far credere attraverso il dire.
Translated row 6/97
Ecco larghe vallate di nubi risolute; e potresti quasi credere che tutto il cielo precipitasse nel mare.
Translated row 7/97
Però, chi spererebbe qualcosa che anche quelli che ancora non credono in Cristo vedono con noi, e perché non possono negarlo, strillan

Jsonl file conversion

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_qwen(few-shot).csv")


jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Translated Sentence": row["Qwen"]
    })


out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-Qwen(few-shot).jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")


# QWEN (TEaR-like)

In [ ]:
csv_path = '/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv'

df = pd.read_csv(csv_path)

The following functions return a list of dictionaries which are compatible with chat template and the text generation pipeline

In [ ]:
def build_translate_prompt(sentence: str) -> List[dict]:
  messages = [
          {
              "role": "system",
              "content": "Sei un traduttore e filologo italiano. Traduci dall'italiano antico (Fiorentino, 1200-1400) all'italiano moderno."
              "- Il significato della frase originale deve essere rispettato senza omissioni o aggiunte improprie.\n"
              "- La frase deve essere grammaticalmente corretta (tempi verbali, punteggiatura, accordi morfologici come genere, numero e concordanza)\n"
              "- Il lessico e la struttura sintattica devono essere naturali e adatti a un lettore contemporaneo italiano.\n"
              "- I termini arcaici e non piu in uso devono essere sostituiti con termini moderni appropriati.\n"
              "- Il tono e lo stile devono essere moderni.\n"
              "- I riferimenti storici, culturali o semantici devono essere interpretati correttamente e adattati al contesto. \n\n"
          },
          {
              "role": "user",
              "content": "Traduci: E però che la seconda suole talora per la grande provedenza fare timoroso, e la prima per l’ardire rendere altrui matto, assai utile è la presente distinzione."
          },
          {
              "role": "assistant",
              "content": "Poiché la seconda, per la sua grande previdenza, può talora incutere timore, e la prima, per l’ardire, condurre alla follia, questa distinzione si rivela assai utile."
          },
          {
              "role": "user",
              "content":  "Traduci: Niuna cosa si dimostra onesta che con suo contrario non si disonesti."
          },
          {
              "role": "assistant",
              "content": "Nessuna cosa si mostra onesta, se non si disonora attraverso il confronto con il suo contrario."
          },
          {
              "role": "user",
              "content":  "Traduci: Era una giovane di maravigliosa bellezza e di costumi onesti, la quale con leggiadria e onestà si portava."
          },
          {
              "role": "assistant",
              "content": "Era una giovane di straordinaria bellezza e di buoni costumi, che si comportava con eleganza e compostezza."
          },
          {
              "role": "user",
              "content":  "Traduci: Diffinizione d'una cosa è dicere ciò che quella cosa è, per tali parole che non si convegnano ad un'altra cosa."
          },
          {
              "role": "assistant",
              "content": "La definizione di una cosa consiste nel dire ciò che essa è, usando parole che non si adattino a un'altra cosa."
          },
          {
              "role": "user",
              "content": f"Traduci: {sentence}"
          },
      ]
  return messages

#Estimate and refinement prompt are used as single-turn prompts so we do not define the system role.
def build_estimate_prompt(source: str, initial_translation: str) -> List[dict]:
    return [{
        "role": "user",
        "content": (
            "Sei un esperto di filologia e traduzione italiana.\n"
            "Fornisci una valutazione critica sulla qualità di una traduzione dall'italiano arcaico (Fiorentino, 1200-1400) all'italiano moderno.\n\n"
            "Presta attenzione ai seguenti aspetti per la valutazione:\n"
            "- Il significato della frase originale è stato rispettato? (niente omissioni o aggiunte improprie)\n"
            "- La frase è grammaticalmente corretta (tempi verbali, punteggiatura, accordi morfologici come genere, numero e concordanza)?\n"
            "- Il lessico e la struttura sintattica sono naturali e adatti a un lettore contemporaneo italiano?\n"
            "- I termini arcaici e non piu in uso sono stati sostituiti con termini moderni appropriati?\n"
            "- Il tono e lo stile sono moderni, scorrevoli e simili a un testo scritto oggi?\n"
            "- Sono stati evitati costrutti rigidi, impersonali o troppo fedeli all’originale antico?\n"
            "- I riferimenti storici, culturali o semantici sono stati interpretati correttamente e adattati al contesto? \n\n"

            f"Frase originale (italiano arcaico): {source}\n"
            f"Traduzione proposta (italiano moderno): {initial_translation}\n"
            f"Valutazione: "
        )
    }]

def build_refine_prompt(source: str, initial_translation: str, feedback: str) -> List[dict]:
    return [{
        "role": "user",
        "content": (
            "Sei un linguista e filologo italiano esperto in traduzione dall'italiano arcaico all'italiano moderno.\n"
            "Dati una frase in italiano arcaico (Fiorentino, 1200-1400), la sua traduzione in italiano moderno e una valutazione della traduzione, migliora la traduzione sulla base della valutazione data.\n\n"
            #f"Frase arcaica (italiano arcaico): {source}\n"
            f"Traduzione iniziale (italiano moderno): {initial_translation}\n"
            f"Valutazione della traduzione:\n{feedback}\n\n"
            "Fornisci SOLO la versione corretta della traduzione basandoti sulla valutazione data senza fornire altre spiegazioni."
        )
    }]


In [ ]:
# Hugging face pipeline in conversational form (text generation)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Lists to save the three prompts outputs
initial_translations = []
feedbacks = []
refined_translations = []

#extract only the assistant reply in the generated block
def extract_assistant_reply(text: str) -> str:
  #divide the text when it find "<|im_start|>assistant"
   #iterate in reverse because we want the last response
    for block in reversed(text.split("<|im_start|>")):
        if block.strip().startswith("assistant"):
           #return clean response without "im_end" and "assistant"
            return block.replace("assistant", "", 1).replace("<|im_end|>", "").strip()
    return text.strip()  #fallback in case it do not find the final assistant block


# Applica la pipeline TEaR a ogni frase
for sentence in tqdm(df["Sentence"]):
    # 1. Translate
    t_translate = build_translate_prompt(sentence)
    #as before (see few shot), we build a compatible template
    prompt_text = tokenizer.apply_chat_template(t_translate, tokenize=False, add_generation_prompt=True)
    #use huggingface pipeline to generate the response
    #it return a list of dictionaries, one for every output
    initial_raw = generator(prompt_text, max_new_tokens=300, do_sample=False)[0]["generated_text"]
    initial = extract_assistant_reply(initial_raw)

    # 2. Estimate
    t_estimate = build_estimate_prompt(sentence, initial)
    prompt_text = tokenizer.apply_chat_template(t_estimate, tokenize=False, add_generation_prompt=True)
    est_raw = generator(prompt_text, max_new_tokens=800, do_sample=False)[0]["generated_text"]
    est = extract_assistant_reply(est_raw)

    # 3. Refine
    t_refine = build_refine_prompt(sentence, initial, est)
    prompt_text = tokenizer.apply_chat_template(t_refine, tokenize=False, add_generation_prompt=True)
    refined_raw = generator(prompt_text, max_new_tokens=1000, do_sample=False)[0]["generated_text"]
    refined = extract_assistant_reply(refined_raw)

    # Save results
    initial_translations.append(initial)
    feedbacks.append(est)
    refined_translations.append(refined)
    print(f"**Initial translation**: {initial}")
    print(f"**Feedback**: {est}")
    print(f"**Final translation**: {refined}")

# Add results in csv file
df["initial_translation"] = initial_translations
df["feedback"] = feedbacks
df["modern_translation"] = refined_translations

df.to_csv("dataset_with_qwen(TEAR).csv", index=False, encoding="utf-8")


  1%|          | 1/97 [00:48<1:18:03, 48.79s/it]

**Initial translation**: Quella guerra era l'opera perché etc. Dall'altra parte, Aiace era un cavaliere francese e valoroso con l'arma, dotato di grande abilità, ma non era ricco di grande saggezza.
**Feedback**: La valutazione critica della traduzione proposta rispetto alla frase originale (italiano arcaico) è la seguente:

1. **Significato**: La traduzione ha mantenuto il significato generale della frase originale, ma manca alcune informazioni chiave. Ad esempio, "l' opera perché" è un'espressione che si riferisce al risultato di una guerra, ma la traduzione non lo chiarisce. Inoltre, "Aiaces" è rimasto invariato, mentre sarebbe più appropriato tradurlo come "Aias", il nome greco corretto. Tuttavia, la frase principale sembra essere stata correttamente tradotta.

2. **Grammatica**: La frase è grammaticalmente corretta, ma manca di alcuni elementi che renderebbero la lingua più fluida e naturale. Per esempio, la punteggiatura potrebbe essere migliorata.

3. **Lessico e struttura sinta

  2%|▏         | 2/97 [01:22<1:03:25, 40.06s/it]

**Initial translation**: Crudele, prende vendetta di tutte le colpe, come stabilisce la legge, e perdonerà a nessun cavaliere che pecca.
**Feedback**: Per valutare questa traduzione, esaminerò attentamente i punti richiesti:

1. **Significato**: Il significato della frase originale è stato mantenuto, ma ci sono alcune sfumature che potrebbero essere state perse. L'originale suggerisce un atteggiamento severo e giudicante verso il peccatore, mentre la traduzione sembra più passiva e neutra. Potrebbe essere utile mantenere l'aspetto più severo dell'originale.

2. **Grammatica**: La frase è grammaticalmente corretta, ma manca di alcuni elementi che rendono la lingua più fluida e naturale. Ad esempio, "come dice la legge" potrebbe essere migliorato con "secondo la legge".

3. **Lessico e struttura sintattica**: Il lessico è adatto a un lettore contemporaneo italiano, ma la struttura sintattica potrebbe essere migliore. La frase sembra un po' troppo semplice e lineare.

4. **Termini arcaici

  3%|▎         | 3/97 [02:02<1:02:53, 40.14s/it]

...

**Initial translation**: Lo pregavo che lo liberasse da quella obbligazione, in cui l'aveva lasciato obbligato. Il gentile uomo accettò e lo liberò, poi fece delle carte con quel foglio.
**Feedback**: La valutazione critica della traduzione proposta dal punto di vista dei criteri richiesti è la seguente:

1. **Significato**: Il significato della frase originale è stato rispettato, ma ci sono alcune sfumature che potrebbero essere state meno chiare nella traduzione. Ad esempio, "in che egli l'aveva lasciato ubbligato" potrebbe essere più chiaro come "in cui l'aveva lasciato obbligato".

2. **Grammatica**: La frase è grammaticalmente corretta, con tempi verbali, punteggiatura e accordi morfologici adeguati. Tuttavia, la struttura sintattica potrebbe essere un po' più fluida per rendere il testo più naturale.

3. **Lessico e struttura sintattica**: Il lessico è adatto a un lettore contemporaneo italiano, ma la struttura sintattica potrebbe essere più semplice e diretta. Ad esempio, "lo li

 99%|█████████▉| 96/97 [1:06:02<00:46, 46.43s/it]

**Initial translation**: L'oro verrà dall'Aquilone. Se consideriamo noi l'Aquilone, non è altro che il popolo gentile congelato dal freddo del peccato, il quale popolo ha sottoposto al suo giogo la tirannia.
**Feedback**: La valutazione critica della traduzione proposta rispetto alla frase originale (italiano arcaico) è la seguente:

1. **Significato**: Il significato della frase originale è stato rispettato. Tuttavia, alcuni elementi sono stati trasformati o modificati in modo che possano essere più facilmente compresi da un lettore moderno. Ad esempio, "L'oro verrà dall'Aquilone" rimane invariato, ma "che figuriamo noi per l'Aquilone" è stata sostituita con "Se consideriamo noi l'Aquilone", che cambia leggermente il senso.

2. **Grammatica**: La frase è grammaticalmente corretta. Tuttavia, la punteggiatura potrebbe essere migliorata per rendere il testo più naturale. Ad esempio, "il quale popolo ha sottoposto al suo giogo la tirannia" potrebbe essere meglio scritto come "il quale pop

100%|██████████| 97/97 [1:06:38<00:00, 41.23s/it]

**Initial translation**: Sia in mezzo tra me e te: tu non puoi più rimanere con noi, se non lo sopporterò e non lo lascerò.
**Feedback**: Per valutare questa traduzione, esamineremo attentamente i punti richiesti:

1. **Significato**: Il significato della frase originale è stato rispettato. Tuttavia, la traduzione manca del "sia in mezzo tra me e te" che è un elemento importante dell'arcaismo. Inoltre, "non lo sofferrò" e "non lo lascerò" potrebbero essere interpretati come un'osservazione passiva, mentre l'originale sembra più direttivo.

2. **Grammatica**: La frase è grammaticalmente corretta. Tuttavia, "sofferrare" è un termine arcaico che non è più utilizzato nel linguaggio quotidiano italiano moderno. Non è stato sostituito con un termine più moderno.

3. **Lessico e struttura sintattica**: Il lessico è naturale e adatto a un lettore contemporaneo italiano. La struttura sintattica è anche adeguata, ma manca del tono e dello stile arcaico.

4. **Termini arcaici**: "Sofferrare" è il

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_qwen(TEAR).csv")

#convert in json
jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Translated Sentence": row["modern_translation"]
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-Qwen(tear).jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")
